In [50]:
import os
import pandas as pd 

os.getcwd()


'/share/data/analyses/christa/colopaint3D/spher_colo52_v1/4_BioImageArchive'

In [51]:

sourceDir = '/share/data/cellprofiler/automation/results'
rootDir = '/share/data/analyses/christa/colopaint3D/spher_colo52_v1/1_Data/'


### Define Functions


In [52]:
metaEx = pd.read_csv(f'{rootDir}spher_colo52-metadata.csv')
these_cols = ['barcode','well_id', 'cmpdname', 'solvent', 'cmpd_conc', 'target', 'pathway', 'pubchemID', 'inkey', 'cell_line', 'article_id']
metaEx = metaEx[these_cols]

# Rename a few columns
metaEx = metaEx.rename(columns={'cmpd_conc': 'cmpd_conc_um', 'article_id': 'selleckchem_id'})

metaEx.head()

,barcode,well_id,cmpdname,solvent,cmpd_conc_um,target,pathway,pubchemID,inkey,cell_line,selleckchem_id
0,PB000137,B02,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036
1,PB000137,B03,Paclitaxel,dmso,0.1,"Autophagy,Microtubule Associated",Cytoskeletal Signaling,441276.0,RCINICONZNJXQF-VAZQATRQSA-N,HCT116,S1150
2,PB000137,B04,"Olaparib (AZD2281, Ku-0059436)",dmso,3.0,PARP,DNA Damage,23725625.0,FDLYAMZZIXQODN-UHFFFAOYSA-N,HCT116,S1060
3,PB000137,B05,SB216763,dmso,10.0,GSK-3,PI3K/Akt/mTOR,176158.0,JCSGFHVFHSKIJH-UHFFFAOYSA-N,HCT116,S1075
4,PB000137,B06,"Vorinostat (SAHA, MK0683)",dmso,3.0,"Autophagy,HDAC",Epigenetics,5311.0,WAEXFXRVDQXREF-UHFFFAOYSA-N,HCT116,S1047


In [53]:
Template = pd.read_csv(f'spher-colo52.tsv', sep='\t')

In [54]:
Template.head()

# Add a new column wiht the barcode (starts with PB0001..)
Template['barcode'] = Template['Files'].str.slice(13, 21)

# # Regex to extract from the filename
Template[['well_id', 'z_plane', 'channel']] = Template['Files'].str.extract(r'Well-([A-Z0-9]{3})-z([0-9]{1,2})-(.*).ome.tiff')

Template.head()



,Files,barcode,well_id,z_plane,channel
0,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,CONC
1,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,HOECHST
2,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,MITO
3,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,PHAandWGA
4,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,SYTO


In [55]:
# Merge the two dataframes on barcode and well_id
FileList = pd.merge(Template, metaEx, on=['barcode', 'well_id'], how='left')

FileList.head()



,Files,barcode,well_id,z_plane,channel,cmpdname,solvent,cmpd_conc_um,target,pathway,pubchemID,inkey,cell_line,selleckchem_id
0,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,CONC,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036
1,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,HOECHST,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036
2,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,MITO,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036
3,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,PHAandWGA,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036
4,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,SYTO,PD0325901,dmso,3.0,MEK,MAPK,9826528.0,SUDAHWBOROXANE-SECBINFHSA-N,HCT116,S1036


In [56]:
# Save the final metadata file
FileList.to_csv(f'spher_colo52-FileList.tsv', sep= '\t',index=False)

#### Now prepare the annotations file

In [57]:
Annotations = pd.read_csv(f'results.tsv', sep='\t')

In [58]:
Annotations

# # Regex to extract from the filename
Annotations[['barcode','Annotation Type','compartment','well_id', 'z_plane']] = Annotations['Files'].str.extract(r'results\/([A-Z0-9]{8})\/([a-z]{12})\/(.*)_.*_([A-Z0-9]{3})_([0-9]{1,2}).npy')

Annotations.loc[Annotations['compartment'] == 'cell', 'channel'] = 'PHAandWGA'
Annotations.loc[Annotations['compartment'] == 'nuclei', 'channel'] = 'HOECHST'

In [63]:
# Replace column "Annotation Type" values "segmentation" with "Segmentation masks"
Annotations['Annotation Type'] = Annotations['Annotation Type'].replace(
    {'segmentation': 'Segmentation masks'})

Annotations


,Files,barcode,Annotation Type,compartment,well_id,z_plane,channel
0,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,0,PHAandWGA
1,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,1,PHAandWGA
2,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,10,PHAandWGA
3,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,11,PHAandWGA
4,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,12,PHAandWGA
...,...,...,...,...,...,...,...
51707,results/PB000142/segmentation/nuclei_segmented...,PB000142,Segmentation masks,nuclei,O23,5,HOECHST
51708,results/PB000142/segmentation/nuclei_segmented...,PB000142,Segmentation masks,nuclei,O23,6,HOECHST
51709,results/PB000142/segmentation/nuclei_segmented...,PB000142,Segmentation masks,nuclei,O23,7,HOECHST
51710,results/PB000142/segmentation/nuclei_segmented...,PB000142,Segmentation masks,nuclei,O23,8,HOECHST


In [64]:
Template.rename(columns={'Files': 'source image'}, inplace=True)

Template

,source image,barcode,well_id,z_plane,channel
0,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,CONC
1,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,HOECHST
2,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,MITO
3,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,PHAandWGA
4,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...,PB000137,B02,0,SYTO
...,...,...,...,...,...
129275,spher-colo52/PB000142_spher-colo52_HT29_L1/Wel...,PB000142,O23,9,CONC
129276,spher-colo52/PB000142_spher-colo52_HT29_L1/Wel...,PB000142,O23,9,HOECHST
129277,spher-colo52/PB000142_spher-colo52_HT29_L1/Wel...,PB000142,O23,9,MITO
129278,spher-colo52/PB000142_spher-colo52_HT29_L1/Wel...,PB000142,O23,9,PHAandWGA


In [65]:
# Merge the two dataframes on barcode and well_id
Annotations_result = pd.merge(Annotations, Template, on=['barcode', 'well_id', 'z_plane', 'channel'], how='left')

Annotations_result.head()

,Files,barcode,Annotation Type,compartment,well_id,z_plane,channel,source image
0,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,0,PHAandWGA,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...
1,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,1,PHAandWGA,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...
2,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,10,PHAandWGA,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...
3,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,11,PHAandWGA,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...
4,results/PB000137/segmentation/cell_segmented_B...,PB000137,Segmentation masks,cell,B02,12,PHAandWGA,spher-colo52/PB000137_spher-colo52_HCT116_L1/W...


In [66]:
# Save the final metadata file
Annotations_result.to_csv(f'spher_colo52-Annotations.tsv', sep= '\t',index=False)